In [339]:
%run ../config/config.py
import pandas as pd
import numpy as np
import debug
import cv2

In [340]:
image = cv2.imread(BLACK_YELLOW_UP)
image_size = (SIZE,SIZE)
image = cv2.resize(image,image_size)

In [341]:
def _detect_edges(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, THRESHOLD1, THRESHOLD2, EDGES)
    return edges

edges_image = _detect_edges(image)

In [342]:
def _generate_edges_lines(canny):
    
    lines = cv2.HoughLines(canny, 
                            rho = RHO_RESOLUTION, 
                            theta =  THETA_RESOLUTION, 
                            threshold = HOUGH_THRESHOLD)
    
    #Format dataset
    lines = lines.reshape(len(lines),2)
    
    coor_lines = []
    for line in lines:
        rho,theta = line
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a * rho
        y0 = b * rho
        x1 = int(x0 +DISTANCE*(-b))
        y1 = int(y0 +DISTANCE*(a))
        x2 = int(x0 -DISTANCE*(-b))
        y2 = int(y0 -DISTANCE*(a))   
        coor_lines.append(((x1,y1),(x2,y2)))
    
    return np.array(coor_lines)
    
lines = _generate_edges_lines(edges_image)

#debug.lines(image, lines, color = RED, size = 4)

In [343]:
from sklearn.cluster import KMeans

def _get_lines_means(points, direction, n_clusters = 9):
    
    means = []

    for lines in points:
    
        kmeans = KMeans(n_clusters = n_clusters, n_init=10)
        kmeans.fit(lines)
        labels = kmeans.labels_
    
        unique_labels = list(range(n_clusters))
    
        cluster_means = []
    
        for label in unique_labels: 
            cluster_data = lines[labels == label] 
            cluster_mean = np.round(sum(cluster_data)/len(cluster_data))  
            cluster_means.append(cluster_mean)
        
        cluster_means = np.array(cluster_means)
        sorted_index = np.argsort(cluster_means[:, direction])
        
        means.append(cluster_means[sorted_index])
   
    #debug.scatter(cluster_means, labels = unique_labels, cmap = 'viridis')
 
    return means


In [344]:


def _get_vertical_averages(lines):
    
    vertical_lines = lines[lines[:,0,1] > SIZE]
    
    initial_points = vertical_lines[:,1][vertical_lines[:,1,1] < -SIZE]
    final_points = vertical_lines[:,0][vertical_lines[:,0,1] > SIZE] 
    
    averages = _get_lines_means((initial_points, final_points), VERTICAL)
    
    return np.stack(averages, axis=1)

def _get_horizontal_averages(lines):

    horizontal_lines = lines[lines[:,1,0] > SIZE]
    
    initial_points = horizontal_lines[:,0][horizontal_lines[:,0,0] < -SIZE]
    final_points = horizontal_lines[:,1][horizontal_lines[:,1,0] > SIZE]
    
    averages = _get_lines_means((initial_points, final_points), HORIZONTAL)
    
    return np.stack(averages, axis=1)
    
vertical_lines = _get_vertical_averages(lines).astype(int)
horizontal_lines = _get_horizontal_averages(lines).astype(int)

lines = np.concatenate((vertical_lines,horizontal_lines), axis=0)

#debug.lines(image,lines, size = 3, color=GREEN);


In [345]:
def _detect_corners(lines, area = 9):

    points = []
    for i in range(area):
        for j in range(area,area*2):

            line1 = lines[i].tolist()
            line2 = lines[j].tolist()
            x_diff = (line1[0][0] - line1[1][0], line2[0][0] - line2[1][0])
            y_diff = (line1[0][1] - line1[1][1], line2[0][1] - line2[1][1])

            def det(a, b):
                return a[0] * b[1] - a[1] * b[0]

            div = det(x_diff, y_diff)

            d = (det(*line1), det(*line2))
            x = det(d, x_diff) / div
            y = det(d, y_diff) / div
            points.append((x, y))

    return points

#debug.lines(image,lines, size = 1);
points = _detect_corners(lines)

points;


In [346]:
def _get_middle_squares(lines):

    middle_lines = []

    for i in range(17):
        
        if i == 8: continue
        
        mean_line = (lines[i] + lines[i+1])/2
        middle_lines.append(mean_line.astype(int))
        
    middle_points = _detect_corners(middle_lines, 8)
    return middle_points

middle_points = _get_middle_squares(lines)
debug.points(image, middle_points, color = BLUE, size = 7)


[(104.88928330457546, 75.52133405936453), (103.85753626966509, 144.27990431446227), (102.72305721984856, 219.8848295629501), (101.52363699016952, 299.81762058370265), (100.31182330162333, 380.5763471132453), (99.08500916421656, 462.33474641328223), (97.91470832551784, 540.3269380208473), (96.89218070412743, 608.471100217794), (180.4042848455792, 77.30260285911659), (179.3994752490825, 145.89923848336727), (178.30108811062362, 220.884253130841), (177.13615963543464, 300.4119312297176), (175.95041655214973, 381.36058708616815), (174.7551917312118, 462.9565449838578), (173.62384370539985, 540.1917431362403), (172.63003248685308, 608.0375382755672), (256.2338134538422, 79.0912908105624), (255.54916583940366, 147.53160055389932), (254.80525692621055, 221.8959236977376), (254.01377163572204, 301.016185414786), (253.20202806906852, 382.16155123847085), (252.38740798553167, 463.59446601774465), (251.62255488665392, 540.0524597234167), (250.94694607029209, 607.5892124733007), (333.3317235365604

In [347]:
def _distance(a, b):
    
    x_diff = abs(a[0] - b[0])
    y_diff = abs(a[1] - b[1])
    
    distance = np.sqrt(x_diff**2 + y_diff**2)
    
    return distance

In [348]:
def _get_grid(middle_points, lines):

    grid_points = np.array(_detect_corners(lines))
    
    grid = []
    
    for middle_point in middle_points:
        
        distances = []
        
        for point in grid_points:
            
            distances.append(_distance(middle_point, point))
            
        index = np.argsort(distances)
        
        corners_distance = grid_points[index]
        
        sqare_corners = corners_distance[:4].tolist()
        
        grid.append(sqare_corners)
        
    return grid

grid = _get_grid(middle_points, lines)

debug.points(image, grid[0], color = GREEN, size = 10)


array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=uint8)

In [349]:

cv2.imshow("Image", image)
cv2.waitKey(0)
cv2.destroyAllWindows()